In [7]:
# library book management system

import tkinter as tk
import tkinter.ttk as ttk
import sqlite3
from tkinter import messagebox

# Create the books table in the database if it doesn't exist
def create_table():
    conn = sqlite3.connect('books.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS books
                      (id INTEGER PRIMARY KEY AUTOINCREMENT,
                       title TEXT NOT NULL,
                       author TEXT NOT NULL,
                       year INTEGER NOT NULL,
                       isbn TEXT)''')
    conn.commit()
    conn.close()

# Clear all input fields
def clear_entries():
    title_entry.delete(0, tk.END)
    author_entry.delete(0, tk.END)
    year_entry.delete(0, tk.END)
    isbn_entry.delete(0, tk.END)

# Display all books in the Treeview
def display_books():
    tree.delete(*tree.get_children())
    conn = sqlite3.connect('books.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM books")
    rows = cursor.fetchall()`
    conn.close()

    for row in rows:
        tree.insert("", tk.END, values=row)

# Insert a new book into the database
def insert_book():
    title = title_entry.get()
    author = author_entry.get()
    try:
        year = int(year_entry.get())
    except ValueError:
        messagebox.showerror("Error", "Year must be a number.")
        return
    isbn = isbn_entry.get()

    if title and author and year:
        conn = sqlite3.connect('books.db')
        cursor = conn.cursor()
        cursor.execute("INSERT INTO books (title, author, year, isbn) VALUES (?, ?, ?, ?)", (title, author, year, isbn))
        conn.commit()
        conn.close()
        clear_entries()
        display_books()
    else:
        messagebox.showerror("Error", "Please fill in all required fields.")

# Delete the selected book
def delete_book():
    selected_item = tree.selection()
    if selected_item:
        book_id = tree.item(selected_item[0], "values")[0]
        conn = sqlite3.connect('books.db')
        cursor = conn.cursor()
        cursor.execute("DELETE FROM books WHERE id=?", (book_id,))
        conn.commit()
        conn.close()
        display_books()
    else:
        messagebox.showerror("Error", "Please select a book to delete.")

# Search for a book by ID
def search_book():
    search_id = search_entry.get()
    try:
        book_id = int(search_id)
    except ValueError:
        messagebox.showerror("Error", "Please enter a valid book ID.")
        return
    
    conn = sqlite3.connect('books.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM books WHERE id = ?", (book_id,))
    rows = cursor.fetchall()
    conn.close()

    tree.delete(*tree.get_children())

    if rows:
        for row in rows:
            tree.insert("", tk.END, values=row)
    else:
        messagebox.showinfo("No Results", "No book found with the specified ID.")
 
# to exit from the window
def close_window():
    window.destroy()

# Create the database table if it doesn't exist
create_table()

# Set up the main Tkinter window
window = tk.Tk()
window.title("Library Management System")

def close_window():
    window.destroy()
    
# Title Frame
title_frame = tk.Frame(window)
title_frame.pack(side=tk.TOP, padx=10, pady=10)
txt_title = tk.Label(title_frame, text='LIBRARY MANAGEMENT SYSTEM', fg='blue', font=('Arial', 16, 'bold'))
txt_title.pack(side=tk.TOP)

# Left Frame for inputs and buttons
left_frame = tk.Frame(window)
left_frame.pack(side=tk.LEFT, padx=10, pady=10)

# Labels and entry fields
tk.Label(left_frame, text="Book Title:").grid(row=0, column=0, padx=5, pady=5, sticky="e")
title_entry = tk.Entry(left_frame)
title_entry.grid(row=0, column=1, padx=5, pady=5)

tk.Label(left_frame, text="Author:").grid(row=1, column=0, padx=5, pady=5, sticky="e")
author_entry = tk.Entry(left_frame)
author_entry.grid(row=1, column=1, padx=5, pady=5)

tk.Label(left_frame, text="Year:").grid(row=2, column=0, padx=5, pady=5, sticky="e")
year_entry = tk.Entry(left_frame)
year_entry.grid(row=2, column=1, padx=5, pady=5)

tk.Label(left_frame, text="ISBN:").grid(row=3, column=0, padx=5, pady=5, sticky="e")
isbn_entry = tk.Entry(left_frame)
isbn_entry.grid(row=3, column=1, padx=5, pady=5)

# Buttons
tk.Button(left_frame, text="Display All", width=15, command=display_books).grid(row=4, column=0, padx=5, pady=5)
tk.Button(left_frame, text="Add Book", width=15, command=insert_book).grid(row=4, column=1, padx=5, pady=5)
tk.Button(left_frame, text="Delete Book", width=15, command=delete_book).grid(row=5, column=0, padx=5, pady=5)
tk.Button(left_frame, text="Exit", width=15, command=close_window).grid(row=5, column=1, padx=5, pady=5)

# Right Frame for table and search
right_frame = tk.Frame(window)
right_frame.pack(side=tk.RIGHT, padx=10, pady=10)

tk.Label(right_frame, text="Search by Book ID:").grid(row=0, column=0, padx=5, pady=5)
search_entry = tk.Entry(right_frame)
search_entry.grid(row=0, column=1, padx=5, pady=5)
tk.Button(right_frame, text="Search", width=10, command=search_book).grid(row=0, column=2, padx=5, pady=5)

# Tree view for displaying books 
tree = ttk.Treeview(right_frame, columns=("ID", "Title", "Author", "Year", "ISBN"), show="headings")
tree.heading("ID", text="ID")
tree.heading("Title", text="Title")
tree.heading("Author", text="Author")
tree.heading("Year", text="Year")
tree.heading("ISBN", text="ISBN")
tree.column("ID", width=30, anchor=tk.CENTER)
tree.column("Title", width=150, anchor=tk.CENTER)
tree.column("Author", width=100, anchor=tk.CENTER)
tree.column("Year", width=60, anchor=tk.CENTER)
tree.column("ISBN", width=100, anchor=tk.CENTER)
tree.grid(row=1, column=0, columnspan=3, padx=10, pady=10)

# Run the Tkinter event loop
window.mainloop()


In [2]:
from sqlite3 import *

def view():
    con = connect('books.db')
    cur = con.cursor()
    cur.execute("SELECT * FROM books")
    rows = cur.fetchall()
    con.close()
    
    return rows


In [3]:
view()

[(3, 'vikas', 'prabhas', 2001, '12012'),
 (6, 'never give up', 'joyce meyer', 2010, '454858'),
 (7, 'Me and myself', 'vikas', 2020, '543322')]